In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

import pandas as pd


url = 'https://www.tokopedia.com/unilever/etalase/discount'
driver = webdriver.Chrome()
driver.get(url)

In [22]:
products = driver.find_elements(By.CLASS_NAME, 'css-1asz3by')

In [23]:
product_list = []

for product in products:
    product_name = product.find_element(By.CSS_SELECTOR, '[data-testid="linkProductName"]').text
    price = product.find_element(By.CSS_SELECTOR, '[data-testid="linkProductPrice"]').text
    disc = product.find_element(By.CLASS_NAME, 'prd_badge-product-discount').text
    try:
        item_solds = product.find_element(By.CLASS_NAME, 'prd_label-integrity').text
    except:
        item_solds = None
    product_dict = {'products': product_name,
                    'price': price,
                    'discount':disc,
                    'item_solds':item_solds}
    product_list.append(product_dict)

In [24]:
page_one_df = pd.DataFrame(product_list)
page_one_df

,products,price,discount,item_solds
0,Lifebuoy Hand Sanitizer Spray Tangan Dan Serba...,Rp26.400,34%,14 terjual
1,Closeup Menthol Fresh 160g isi 2 FREE Pepsoden...,Rp34.900,63%,80+ terjual
2,Pepsodent Pasta Gigi Gum Expert Whitening 100g...,Rp59.300,54%,30+ terjual
3,CIF Cream Pembersih Serbaguna Lemon Botol 660g...,Rp33.400,66%,30+ terjual
4,Rexona Women Anti-Perspirant Deodorant Roll On...,Rp51.100,41%,30+ terjual
5,Rexona Deodorant Deo Roll On Advanced Whitenin...,Rp64.300,40%,23 terjual
6,Sunlight Sabun Cuci Piring Extra Lembut 650mL ...,Rp26.500,37%,250+ terjual
7,Rinso Molto Detergent Bubuk Deterjen Anti Noda...,Rp49.200,37%,13 terjual
8,Sariwangi Hitam Manis Asli 1.85G (30pcs) isi 3,Rp20.300,38%,13 terjual
9,Dove Shampo Perawatan Rambut Rontok Sampo 430m...,Rp110.200,34%,3 terjual


In [25]:
for page in range(2, 51):
    url = f'https://www.tokopedia.com/unilever/etalase/discount/page/{page}'
    driver.get(url)
    time.sleep(13)
    products = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'css-1asz3by'))
)
    
    for product in products:
        try:
            product_name = product.find_element(By.CSS_SELECTOR, '[data-testid="linkProductName"]').text
        except:
            product_name = None
        
        try:
            price = product.find_element(By.CSS_SELECTOR, '[data-testid="linkProductPrice"]').text
        except:
            price = None
        
        try:
            disc = product.find_element(By.CLASS_NAME, 'prd_badge-product-discount').text
        except:
            disc = None
        
        try:
            item_solds = product.find_element(By.CLASS_NAME, 'prd_label-integrity').text
        except:
            item_solds = None

        product_dict = {'products': product_name,
                        'price': price,
                        'discount': disc,
                        'item_solds': item_solds}
        product_list.append(product_dict)

data = pd.DataFrame(product_list)
data.shape

(780, 4)

In [26]:
data = pd.concat([page_one_df, data])
data.head()

,products,price,discount,item_solds
0,Lifebuoy Hand Sanitizer Spray Tangan Dan Serba...,Rp26.400,34%,14 terjual
1,Closeup Menthol Fresh 160g isi 2 FREE Pepsoden...,Rp34.900,63%,80+ terjual
2,Pepsodent Pasta Gigi Gum Expert Whitening 100g...,Rp59.300,54%,30+ terjual
3,CIF Cream Pembersih Serbaguna Lemon Botol 660g...,Rp33.400,66%,30+ terjual
4,Rexona Women Anti-Perspirant Deodorant Roll On...,Rp51.100,41%,30+ terjual


In [27]:
data.isnull().sum()

products      0
price         0
discount      0
item_solds    7
dtype: int64

In [28]:
data['discount']=data['discount'].apply(lambda x: x.replace('%','')).astype('float')
data['price']=data['price'].apply(lambda x: x.replace('Rp','').replace('.','')).astype('float')
data

,products,price,discount,item_solds
0,Lifebuoy Hand Sanitizer Spray Tangan Dan Serba...,26400.0,34.0,14 terjual
1,Closeup Menthol Fresh 160g isi 2 FREE Pepsoden...,34900.0,63.0,80+ terjual
2,Pepsodent Pasta Gigi Gum Expert Whitening 100g...,59300.0,54.0,30+ terjual
3,CIF Cream Pembersih Serbaguna Lemon Botol 660g...,33400.0,66.0,30+ terjual
4,Rexona Women Anti-Perspirant Deodorant Roll On...,51100.0,41.0,30+ terjual
...,...,...,...,...
775,Tresemme Keratin Smooth Shampoo 340Ml Free Pen...,72800.0,7.0,60+ terjual
776,Pond's Berry Glow Facial Mask 8G Free Matcha F...,14100.0,57.0,250+ terjual
777,Pond's Matcha Clear Facial Mask 8G Free Berry ...,14100.0,57.0,100+ terjual
778,Wipol Spray Disinfektan Power Clean 500Ml Free...,19900.0,42.0,80+ terjual


In [29]:
data['item_solds']=data['item_solds'].fillna('0 terjual')

In [30]:
data['item_solds']=data['item_solds'].apply(lambda x: x.replace('terjual','').replace('+',''))

In [31]:
data['item_solds'].unique()

array(['14 ', '80 ', '30 ', '23 ', '250 ', '13 ', '3 ', '1 ', '17 ',
       '18 ', '7 ', '6 ', '2 ', '4 ', '0 ', '5 ', '100 ', '50 ', '40 ',
       '90 ', '27 ', '11 ', '10 ', '3rb ', '500 ', '1rb ', '750 ', '25 ',
       '15 ', '20 ', '24 ', '29 ', '16 ', '70 ', '8 ', '12 ', '2rb ',
       '9 ', '60 ', '5rb ', '26 ', '22 ', '6rb ', '19 ', '10rb ', '50rb ',
       '4rb ', '9rb ', '21 ', '28 '], dtype=object)

In [32]:
data['item_solds']=data['item_solds'].apply(lambda x: x.replace('rb','000').replace('+',''))

In [33]:
data.to_csv("unilever-tokopedia-scrapped_1-20")